# Covid-19 Corpus Building Project Group G

## **Pragmatics of COVID-19 related Mental Health in New Zealand and Australian - Newspapers VS. Personal Accounts**

### **CONTENT**:
1. Import required python libraries

2. Newspaper Files Corpus

3. Personal Perspective Corpus (Personal Stories + Additional Comments)

## **Import required python libraries**

In [2]:
import pandas as pd
# operating file system
import os
# reqular expressions for recording text rules
import re 
# conver HTML to text
from bs4 import BeautifulSoup
# avaliable for web requested
import requests 
# zip corpus
import zipfile
# Pausing the requests
import datetime
# Finding file directories and files and return the results of the search to a list
import glob

In [21]:
os.getcwd()

'/home/pwu17/Assignment 1'

## **Newspaper Files Corpus**

### **Convert Factiva HTML to Corpus Text Files  (for news database)**

In [22]:
# factiva html directory
directory_with_factivahtml = 'Factiva_html/'
# output corpus directory
directory_to_output_corpus = 'newspaper_text_files/'

In [23]:
factivafiles = []
for f in glob.glob(directory_with_factivahtml + "*.html"):
    factivafiles.append(f)
factivafiles

['Factiva_html/Factiva aus.html', 'Factiva_html/Factiva.html']

In [84]:
# This stage shows details about each text file and extract text from the html file
for factivafile in factivafiles:
    soup = BeautifulSoup(open(factivafile, encoding='utf8'), "html.parser") 
    i = 0
    for article in soup.select(".article .article,#lastArticle"):
        
        i += 1
        print('---------------' + factivafile + '|' + str(i) + '-----------------')
        doc_string = article.find_all('p',attrs={'class': None},text=re.compile('^Document ')) 
        print('doc#:',doc_string[0].get_text())
        for headline in article.select("#hd"):
            print('headline:',headline.get_text().strip())
            body = headline.get_text().strip() + "\n"
        for author in article.select("div.author"):
            print('author:',author.get_text())
        words_string = article.find_all('div',attrs={'class': None},text=re.compile('[0-9]{1,2} words')) 
        print('words:',words_string[0].get_text())
        date_string = article.find_all('div',attrs={'class': None},text=re.compile('[0-9]{1,2} [A-Za-z]{1,} [0-9]{4}')) 
        print('date:',date_string[0].get_text())
        
        time_string = article.find_all('div',attrs={'class': None},text=re.compile('[0-9]{2}:[0-9]{2}')) 
        
        if (len(time_string) > 0):
            print('time:',time_string[0].get_text())
            print('publisher: ', time_string[0].next_sibling.get_text()) 
            short_publisher_id = time_string[0].next_sibling.next_sibling.get_text()
        else:
            print('publisher: ', date_string[0].next_sibling.get_text())
            short_publisher_id = date_string[0].next_sibling.next_sibling.get_text()
        print('short publisher id: ', short_publisher_id) #short publisher
        
        copyright_string = article.find_all('div',attrs={'class': None},text=re.compile('.*(All Rights Reserved|\(c\)|Copyright|©).*',flags=re.IGNORECASE)) 
        language = ''
        if (len(copyright_string) > 0):
            print('copyright:',copyright_string[0].get_text())
            language = copyright_string[0].previous_sibling.get_text()
            print('Language:',copyright_string[0].previous_sibling.get_text())
        else:
            print('Warning: No language!!!!!')

        for paragraph in article.select(".articleParagraph"):
            body += paragraph.get_text().strip() + "\n"

        format_str = '%d %B %Y'
        datetime_obj = datetime.datetime.strptime(date_string[0].get_text(), format_str)
        if (len(time_string) > 0):
            
            fileid = str(datetime_obj.date()) + '-' + time_string[0].next_sibling.get_text() + '-' + headline.get_text().strip()
        else:
            fileid = str(datetime_obj.date()) + '-' + date_string[0].next_sibling.get_text() + '-' + headline.get_text().strip()
            with open(directory_to_output_corpus + fileid + '.txt', 'w', encoding='utf8') as f:
                f.write(body)
                f.close()
                print('Wrote file: ',directory_to_output_corpus + fileid + '.txt')

---------------Factiva_html/Factiva aus.html|1-----------------
doc#: Document DAITEL0020220821ei8l0005i
headline: PAINE DESERVED A LOT BETTER
author: Ben Horne BBL CRICKET 
words: 769 words
date: 21 August 2022
publisher:  Daily Telegraph
short publisher id:  DAITEL
copyright: © News Pty Limited. No redistribution is permitted. 
Language: English
Wrote file:  newspaper_text_files/2022-08-21-Daily Telegraph-PAINE DESERVED A LOT BETTER.txt
---------------Factiva_html/Factiva aus.html|2-----------------
doc#: Document AUSTLN0020220819ei8k00036
headline: Why coaching has become the most extreme sport
author: PETER LALOR 
words: 2178 words
date: 20 August 2022
publisher:  The Australian
short publisher id:  AUSTLN
copyright: © News Pty Limited. No redistribution is permitted. 
Language: English
Wrote file:  newspaper_text_files/2022-08-20-The Australian-Why coaching has become the most extreme sport.txt
---------------Factiva_html/Factiva aus.html|3-----------------
doc#: Document AUSMAG00

###  **Convert CSV files to Text Files  (for newspaper webs)**

In [3]:
with open('stuff_articles.csv', encoding='utf-8') as a:  
    df_s = pd.read_csv(a) 
df_s.head(10) 

,title,date,story_text,web-scraper-order,web-scraper-start-url,link,link-href
0,"The more lockdowns Auckland is placed under, t...","05:00, Sep 08 2021","The more lockdowns Auckland is placed under, t...",1661240483-261,https://www.stuff.co.nz/searchresults?q=covid+...,Covid-19: Mental health 'suffers more every lo...,https://www.stuff.co.nz/national/health/corona...
1,Mental health conditions amongst Kiwi youth ha...,"05:00, Sep 03 2020",Mental health conditions amongst Kiwi youth ha...,1661240472-260,https://www.stuff.co.nz/searchresults?q=covid+...,Youth mental health problems double in 10 year...,https://www.stuff.co.nz/national/health/300097...
2,The people leading our Covid response deserve ...,"05:00, Aug 29 2021",The people leading our Covid response deserve ...,1661240563-268,https://www.stuff.co.nz/searchresults?q=covid+...,"During Covid, spare a thought for our leaders'...",https://www.stuff.co.nz/national/health/corona...
3,"Waikato schools are closed, and how they opera...","15:59, Aug 18 2021","Waikato schools are closed, and how they opera...",1661240540-266,https://www.stuff.co.nz/searchresults?q=covid+...,Covid-19: Mental health at the forefront of sc...,https://www.stuff.co.nz/national/education/126...
4,Don’t tough it out – talk to people and try to...,"13:30, Aug 19 2021",Don’t tough it out – talk to people and try to...,1661240551-267,https://www.stuff.co.nz/searchresults?q=covid+...,Covid-19: Mental health toll on Kiwis as NZ go...,https://www.stuff.co.nz/national/health/corona...
5,A surge in child mental health cases is expect...,"05:00, Mar 22 2022",A surge in child mental health cases is expect...,1661240495-262,https://www.stuff.co.nz/searchresults?q=covid+...,Doctor fears new mental health crisis for chil...,https://www.stuff.co.nz/national/health/128115...
6,Kiwis are turning to online therapies as deman...,"07:47, Nov 05 2021",Kiwis are turning to online therapies as deman...,1661240460-259,https://www.stuff.co.nz/searchresults?q=covid+...,Ongoing Covid lockdowns cause spike in demand ...,https://www.stuff.co.nz/national/health/126864...
7,National health spokesman Shane Reti says he a...,"14:46, Mar 07 2022",National health spokesman Shane Reti says he a...,1661240517-264,https://www.stuff.co.nz/searchresults?q=covid+...,Covid-19 NZ: Shane Reti article on mental heal...,https://www.stuff.co.nz/national/politics/3005...
8,A new report warns of a mental health tsunami ...,"17:44, Jun 13 2020",A new report warns of a mental health tsunami ...,1661240528-265,https://www.stuff.co.nz/searchresults?q=covid+...,Mental health needs set to double in wake of C...,https://www.stuff.co.nz/national/health/121820...
9,"""It's OK to not be OK"" – that's a clinical psy...","17:47, Mar 30 2020","""It's OK to not be OK"" – that's a clinical psy...",1661240506-263,https://www.stuff.co.nz/searchresults?q=covid+...,Coronavirus and mental health: 'It's OK to not...,https://www.stuff.co.nz/life-style/well-good/i...


In [5]:
# Reading and Writing csv file
text_col = 'story_text'

for idx, col in df_s.iterrows():   
    if isinstance(col[text_col],str):
        filename = 'newspaper_text_files/{}.txt'.format(col['title'][:35] + '-' + col['date'])
        with open(filename, 'w', encoding='utf-8') as a:
       
            a.write(col[text_col])
            print('Writing file ' + str(idx), filename)
    else:
        print('No string data - ignoring row',idx)

Writing file 0 newspaper_text_files/The more lockdowns Auckland is plac-05:00, Sep 08 2021.txt
Writing file 1 newspaper_text_files/Mental health conditions amongst Ki-05:00, Sep 03 2020.txt
Writing file 2 newspaper_text_files/The people leading our Covid respon-05:00, Aug 29 2021.txt
Writing file 3 newspaper_text_files/Waikato schools are closed, and how-15:59, Aug 18 2021.txt
Writing file 4 newspaper_text_files/Don’t tough it out – talk to people-13:30, Aug 19 2021.txt
Writing file 5 newspaper_text_files/A surge in child mental health case-05:00, Mar 22 2022.txt
Writing file 6 newspaper_text_files/Kiwis are turning to online therapi-07:47, Nov 05 2021.txt
Writing file 7 newspaper_text_files/National health spokesman Shane Ret-14:46, Mar 07 2022.txt
Writing file 8 newspaper_text_files/A new report warns of a mental heal-17:44, Jun 13 2020.txt
Writing file 9 newspaper_text_files/"It's OK to not be OK" – that's a c-17:47, Mar 30 2020.txt


In [7]:
# zip output files
import zipfile
corpus_pa = 'newspaper_text_files/'

zip_filename = os.path.dirname(corpus_pa) + '.zip'

zipf = zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED)

# ziph is zipfile handle
for root, dirs, files in os.walk(corpus_pa):
    for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(corpus_pa, '..')))
      
zipf.close()

## **Personal Perspectives Corpus**

### **Extracting Text Files from CSV files (for personal stories)**

In [14]:
with open('Opinions_corpus.csv', encoding='utf-8') as f:  
    df = pd.read_csv(f) 
df.head(10) 

,date,title,url,body,description
0,01/06/2020,1661045120-31228,https://mentalhealth.org.nz/search?query=covid...,Emma Campbell has been in the ambulance servic...,Working as an ambulance officer during a pandemic
1,01/07/2020,1661045111-31226,https://mentalhealth.org.nz/search?query=covid...,When we asked for nominations for Awesome Auck...,BBM foodbank supporting thousands of wh?nau th...
2,01/08/2020,1661045116-31227,https://mentalhealth.org.nz/search?query=covid...,Nicky Primrose has worked with Mitre 10 for fi...,"Mitre 10 employs 6,000 people in New Zealand –..."
3,01/09/2020,1661045098-31223,https://mentalhealth.org.nz/search?query=covid...,It would be an understatement to say that the ...,The day before New Zealand moved into level fo...
4,01/10/2020,1661045107-31225,https://mentalhealth.org.nz/search?query=covid...,"Dr Jeff Garon is a GP at ?r?kei Health, a wh?n...",Supporting the community during a pandemic – o...
5,01/11/2020,1661045102-31224,https://mentalhealth.org.nz/search?query=covid...,Eteroa Lafaele first moved to Auckland for uni...,NaN
6,01/06/2020,1660953337-22,https://allsorts.org.nz/?gclid=CjwKCAjw6fyXBhB...,"“In all honesty, I was really apprehensive, an...",Student flatmates experiencing COVID-19
7,01/06/2020,1660953345-33,https://allsorts.org.nz/?gclid=CjwKCAjw6fyXBhB...,"“Call your family, call your friends, and call...",University student Keilan Rochette arrived in ...
8,01/06/2020,1660953332-15,https://allsorts.org.nz/?gclid=CjwKCAjw6fyXBhB...,George’s partner Teah talks about his ability ...,A lay vaccinator tackling vaccine hesitancy wi...
9,01/06/2020,1660953337-25,https://allsorts.org.nz/?gclid=CjwKCAjw6fyXBhB...,"“Don’t be hard on yourself,” Sammy says. “I kn...",Student flatmates experiencing COVID-19


In [25]:
df.shape[0]

123

In [22]:
text_col = 'body'

for idx, col in df.iterrows():   
    if isinstance(col[text_col],str):
        filename = 'personal_story_text_files/{}.txt'.format(str(idx))
        with open(filename, 'w', encoding='utf-8') as f:
       
            f.write(col[text_col])
            print('Writing file ' + str(idx), filename)
    else:
        print('No string data - ignoring row',idx)
        

Writing file 0 personal_story_text_files/0.txt
Writing file 1 personal_story_text_files/1.txt
Writing file 2 personal_story_text_files/2.txt
Writing file 3 personal_story_text_files/3.txt
Writing file 4 personal_story_text_files/4.txt
Writing file 5 personal_story_text_files/5.txt
Writing file 6 personal_story_text_files/6.txt
Writing file 7 personal_story_text_files/7.txt
Writing file 8 personal_story_text_files/8.txt
Writing file 9 personal_story_text_files/9.txt
Writing file 10 personal_story_text_files/10.txt
Writing file 11 personal_story_text_files/11.txt
Writing file 12 personal_story_text_files/12.txt
Writing file 13 personal_story_text_files/13.txt
Writing file 14 personal_story_text_files/14.txt
Writing file 15 personal_story_text_files/15.txt
Writing file 16 personal_story_text_files/16.txt
Writing file 17 personal_story_text_files/17.txt
Writing file 18 personal_story_text_files/18.txt
Writing file 19 personal_story_text_files/19.txt
Writing file 20 personal_story_text_file

In [26]:
# zip output files
import zipfile
corpus_path_1 = 'personal_story_text_files/'
zip_filename = os.path.dirname(corpus_path_1) + '.zip'

zipf = zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED)

# ziph is zipfile handle
for root, dirs, files in os.walk(corpus_path_1):
    for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(corpus_path_1, '..')))
      
zipf.close()

### **Convert csv file to text file (for some comments on social media)**

In [15]:
with open('Additional reddit corpus.csv') as f:  # csv files for comments text content
    df = pd.read_csv(f) 
df.head(10)

,date,scraper ref,url,body
0,pre 01.01.2021,1661058856-33176,https://redditsearchtool.com/?subreddit=Austra...,Your whole argument against lockdowns is summe...
1,pre 01.01.2021,1661058856-33135,https://redditsearchtool.com/?subreddit=Austra...,"Thank you for the advice, I appreciate it. I d..."
2,pre 01.01.2021,1661058856-33142,https://redditsearchtool.com/?subreddit=Austra...,https://agedcare.royalcommission.gov.au/news-a...
3,pre 01.01.2021,1661058856-33171,https://redditsearchtool.com/?subreddit=Austra...,I think our country is wonderful and our peopl...
4,pre 01.01.2021,1661058856-33163,https://redditsearchtool.com/?subreddit=Austra...,Certainly some of this list contains genuine c...
5,pre 01.01.2021,1661058856-33111,https://redditsearchtool.com/?subreddit=Austra...,I 100% agree in fact I am a firm believer that...
6,pre 01.01.2021,1661058856-33150,https://redditsearchtool.com/?subreddit=Austra...,Can we stop glorifying the guy who defended se...
7,pre 01.01.2021,1661058856-33175,https://redditsearchtool.com/?subreddit=Austra...,"Everyone here loves to go on about ""go get you..."
8,pre 01.01.2021,1661058856-33080,https://redditsearchtool.com/?subreddit=Austra...,"The issue is more what it didn’t explore, name..."
9,pre 01.01.2021,1661058856-33159,https://redditsearchtool.com/?subreddit=Austra...,I actually think that people will have more fa...


In [16]:
text_col = 'body'

for idx, col in df.iterrows():   
    if isinstance(col[text_col],str):
        filename = 'comments_text_files/{}.txt'.format(col['scraper ref'][:400] )
        with open(filename, 'w', encoding='utf-8') as f:
       
            f.write(col[text_col])
            print('Writing file ' + str(idx), filename)
    else:
        print('No string data - ignoring row',idx)

Writing file 0 comments_text_files/1661058856-33176.txt
Writing file 1 comments_text_files/1661058856-33135.txt
Writing file 2 comments_text_files/1661058856-33142.txt
Writing file 3 comments_text_files/1661058856-33171.txt
Writing file 4 comments_text_files/1661058856-33163.txt
Writing file 5 comments_text_files/1661058856-33111.txt
Writing file 6 comments_text_files/1661058856-33150.txt
Writing file 7 comments_text_files/1661058856-33175.txt
Writing file 8 comments_text_files/1661058856-33080.txt
Writing file 9 comments_text_files/1661058856-33159.txt
Writing file 10 comments_text_files/1661058856-33134.txt
Writing file 11 comments_text_files/1661058856-33093.txt
Writing file 12 comments_text_files/1661058856-33119.txt
Writing file 13 comments_text_files/1661058856-33162.txt
Writing file 14 comments_text_files/1661058856-33152.txt
Writing file 15 comments_text_files/1661058856-33083.txt
Writing file 16 comments_text_files/1661058856-33166.txt
Writing file 17 comments_text_files/16610

In [17]:
# zipping the output file folder
corpus_path_2 = 'comments_text_files/'
zip_filename = os.path.dirname(corpus_path_2) + '.zip'

zipf = zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED)

# ziph is zipfile handle
for root, dirs, files in os.walk(corpus_path_2):
    for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(corpus_path_2, '..')))
      
zipf.close()